In [1]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

In [2]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import RidgeClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC
from sklearn.tree import ExtraTreeClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors.nearest_centroid import NearestCentroid
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import pandas as pd
import time
import numpy as np

In [3]:
from ensembles import DiversityEnsembleClassifier as DEC
from ensembles import GeneticEnsembleClassifier as GEC
from ensembles import RandomClassifier as RC
from ensembles import MajorityClassifier as MC

### Loading datasets

In [5]:
breast = datasets.load_breast_cancer()
iris   = datasets.load_iris()
wine   = datasets.load_wine()
yeast  = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/yeast/yeast.data', header=None, delim_whitespace=True)
le = LabelEncoder()
yeast.iloc[:, -1] = le.fit_transform(yeast.iloc[:, -1])
#spam   = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/spambase/spambase.data', header=None)
captcha = pd.read_csv('dados_captcha.csv')

In [6]:
captcha.shape

(2788, 78)

<h3>Testing method</h3>

In [12]:
def test_algorithm(classifier, data, target):
    accuracy, f1, precision, recall, auc = 0, 0, 0, 0, 0
    div = np.zeros(25)
    fit = np.zeros(25)
    for i in range(0, 10):
        print(i, end='')
        X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2, random_state=i*10)
        try:classifier.set_param(random_state=i*10)
        except: classifier.random_state=i*10
        aux = classifier.fit(X_train, y_train)
        try: 
            div += aux[0]
            fit += aux[1]
        except: pass
        y_pred = classifier.predict(X_test)
        accuracy += accuracy_score(y_test, y_pred)
        try: f1 += f1_score(y_test, y_pred)
        except: pass
        try: precision += precision_score(y_test, y_pred)
        except: pass
        try: recall += recall_score(y_test, y_pred)
        except: pass
        try: auc += roc_auc_score(y_test, y_pred)
        except: pass
    print('Accuracy :', accuracy/10)
    if f1>0: print('F1-score :', f1/10)
    if precision>0: print('Precision:', precision/10)
    if recall>0: print('Recall   :', recall/10)
    if auc>0: print('ROC AUC  :', auc/10)
    return div/10, fit/10

In [13]:
def compare_results(data, target, size):
    n_samples = ((data.shape[0]*0.8 * 4) // 5)-1
    alg = {
                KNeighborsClassifier: {'n_neighbors':[1, n_samples]},
                RidgeClassifier: {'alpha':[1.0, 10.0],'max_iter':[100, 1000]},
                SVC: {'C':[1, 1000],
                      'gamma':[0.0001, 0.001]
                      },
                DecisionTreeClassifier: {'min_samples_leaf':[1, n_samples], 'max_depth':[1, n_samples]},
                ExtraTreeClassifier: {'min_samples_leaf':[1, n_samples], 'max_depth':[1, n_samples]},
                GaussianNB: {},
                LinearDiscriminantAnalysis: {},
                QuadraticDiscriminantAnalysis: {},
                BernoulliNB: {},
                GaussianProcessClassifier: {'max_iter_predict':[100, 1000]},
                LogisticRegression:{'C':[1, 1000], 'max_iter':[100, 1000]},
                NearestCentroid:{},
                PassiveAggressiveClassifier:{'C':[1, 1000], 'max_iter':[100, 1000]},
                SGDClassifier: {'alpha':[1e-5, 1e-2], 'max_iter':[100, 1000]}
          }
        
    print('-'*60)
    print('Diversity-based Ensemble Classifier')
    print('-'*60)
    aux = int(round(time.time() * 1000))
    dec = DEC(alg, population_size=size, max_epochs=20, random_state=42)
    div, fit = test_algorithm(dec, data=data, target=target)
    print('DEC done in: ',(int(round(time.time() * 1000)) - aux)/10, 'ms')
    
    print('-'*60)
    print('Genetic-based Ensemble Classifier')
    print('-'*60)
    aux = int(round(time.time() * 1000))
    gec = GEC(alg, population_size=size, max_epochs=20, random_state=42)
    test_algorithm(gec, data=data, target=target)
    print('GEC done in: ',(int(round(time.time() * 1000)) - aux)/10, 'ms')    
                   
    print('-'*60)
    print('Random Classifier')
    print('-'*60)
    aux = int(round(time.time() * 1000))
    rc = RC(alg, random_state=42)
    test_algorithm(rc, data=data, target=target)
    print('RC done in: ',(int(round(time.time() * 1000)) - aux)/10, 'ms')
      
    print('-'*60)
    print('Majority Classifier')
    print('-'*60)
    aux = int(round(time.time() * 1000))
    mc = MC()
    test_algorithm(mc, data=data, target=target)
    print('MC done in: ',(int(round(time.time() * 1000)) - aux)/10, 'ms')
        
    print('-'*60)
    print('Random Forest')
    print('-'*60)
    aux = int(round(time.time() * 1000))
    rf = RandomForestClassifier(n_estimators=size, random_state=42)
    test_algorithm(rf, data=data, target=target)
    print('RF done in: ',(int(round(time.time() * 1000)) - aux)/10, 'ms')
        
    print('-'*60)
    print('Ada Boost')
    print('-'*60)
    aux = int(round(time.time() * 1000))
    ada = AdaBoostClassifier(n_estimators=size, random_state=42)
    test_algorithm(ada, data=data, target=target)
    print('Ada done in: ',(int(round(time.time() * 1000)) - aux)/10, 'ms')
       
    return div, fit

In [ ]:
d10, f10   =compare_results(data=captcha.iloc[:, 1:-1].values, target=captcha.iloc[:, -1].values, size=10)
d25, f25   =compare_results(data=captcha.iloc[:, 1:-1].values, target=captcha.iloc[:, -1].values, size=25)
d50, f50 =compare_results(data=captcha.iloc[:, 1:-1].values, target=captcha.iloc[:, -1].values, size =50)

plt.plot(range(1, 26), d10, label='10 cromossomos')
plt.plot(range(1, 26), d25, label='25 cromossomos')
plt.plot(range(1, 26), d50, label='50 cromossomos')
plt.legend()
plt.ylabel('Diversidade')
plt.xlabel('Nº de épocas')
plt.grid(ls='dashed')
plt.savefig('captcha.png')
plt.clf()
plt.close()

plt.plot(range(1, 26), f10, label='10 cromossomos')
plt.plot(range(1, 26), f25, label='25 cromossomos')
plt.plot(range(1, 26), f50, label='50 cromossomos')
plt.legend()
plt.ylabel('Fitness')
plt.xlabel('Nº de épocas')
plt.grid(ls='dashed')
plt.savefig('captchaFitness.png')
plt.clf()
plt.close()

------------------------------------------------------------
Diversity-based Ensemble Classifier
------------------------------------------------------------
0

In [ ]:
d25, f25   =compare_results(data=breast.data, target=breast.target, size=25)
d50, f50   =compare_results(data=breast.data, target=breast.target, size=50)
d100, f100 =compare_results(data=breast.data, target=breast.target, size =100)

plt.plot(range(1, 101), d25, label='25 cromossomos')
plt.plot(range(1, 101), d50, label='50 cromossomos')
plt.plot(range(1, 101), d100, label='100 cromossomos')
plt.legend()
plt.ylabel('Diversidade')
plt.xlabel('Nº de épocas')
plt.grid(ls='dashed')
plt.savefig('breast2.png')
plt.clf()
plt.close()

plt.plot(range(1, 101), f25, label='25 cromossomos')
plt.plot(range(1, 101), f50, label='50 cromossomos')
plt.plot(range(1, 101), f100, label='100 cromossomos')
plt.legend()
plt.ylabel('Fitness')
plt.xlabel('Nº de épocas')
plt.grid(ls='dashed')
plt.savefig('breastFitness.png')
plt.clf()
plt.close()

In [ ]:
d25, f25 = compare_results(data=iris.data, target=iris.target, size=25)

d50, f50 = compare_results(data=iris.data, target=iris.target, size=50)

d100, f100 = compare_results(data=iris.data, target=iris.target, size=100)

plt.plot(range(1, 21), d25, label='25 cromossomos')
plt.plot(range(1, 21), d50, label='50 cromossomos')
plt.plot(range(1, 21), d100, label='100 cromossomos')
plt.legend()
plt.ylabel('Diversidade')
plt.xlabel('Nº de épocas')
plt.grid(ls='dashed')
plt.savefig('iris.png')
plt.clf()
plt.close()


plt.plot(range(1, 21), f25, label='25 cromossomos')
plt.plot(range(1, 21), f50, label='50 cromossomos')
plt.plot(range(1, 21), f100, label='100 cromossomos')
plt.legend()
plt.ylabel('Fitness')
plt.xlabel('Nº de épocas')
plt.grid(ls='dashed')
plt.savefig('irisFitness.png')
plt.clf()
plt.close()

In [9]:
d25, f25 = compare_results(data=wine.data, target=wine.target, size=25)
d50, f50 = compare_results(data=wine.data, target=wine.target, size=50)
d100, f100 = compare_results(data=wine.data, target=wine.target, size=100)

plt.plot(range(1, 51), d25, label='25 cromossomos')
plt.plot(range(1, 51), d50, label='50 cromossomos')
plt.plot(range(1, 51), d100, label='100 cromossomos')
plt.legend()
plt.ylabel('Diversidade')
plt.xlabel('Nº de épocas')
plt.grid(ls='dashed')
plt.savefig('wine.png')
plt.clf()
plt.close()

plt.plot(range(1, 51), f25, label='25 cromossomos')
plt.plot(range(1, 51), f50, label='50 cromossomos')
plt.plot(range(1, 51), f100, label='100 cromossomos')
plt.legend()
plt.ylabel('Fitness')
plt.xlabel('Nº de épocas')
plt.grid(ls='dashed')
plt.savefig('wineFitness.png')
plt.clf()
plt.close()

------------------------------------------------------------
Diversity-based Ensemble Classifier
------------------------------------------------------------
Accuracy : 0.8666666666666668
DEC done in:  20594.8 ms
------------------------------------------------------------
Genetic-based Ensemble Classifier
------------------------------------------------------------
Accuracy : 0.986111111111111
GEC done in:  19775.3 ms
------------------------------------------------------------
Random Classifier
------------------------------------------------------------
Accuracy : 0.31111111111111106
RC done in:  0.9 ms
------------------------------------------------------------
Random Forest
------------------------------------------------------------
Accuracy : 0.9694444444444443
RF done in:  38.6 ms
------------------------------------------------------------
Ada Boost
------------------------------------------------------------
Accuracy : 0.8777777777777777
Ada done in:  41.9 ms
---------------

KeyboardInterrupt: 

In [10]:
d25 = compare_results(data=yeast.iloc[:, 1:-1].values, target=yeast.iloc[:, -1].values, size = 25)
d50 = compare_results(data=yeast.iloc[:, 1:-1].values, target=yeast.iloc[:, -1].values, size = 50)
d100 = compare_results(data=yeast.iloc[:, 1:-1].values, target=yeast.iloc[:, -1].values, size = 100)

plt.plot(range(1, 101), d25, label='25 cromossomos')
plt.plot(range(1, 101), d50, label='50 cromossomos')
plt.plot(range(1, 101), d100, label='100 cromossomos')
plt.legend()
plt.ylabel('Diversidade')
plt.xlabel('Nº de épocas')
plt.grid(ls='dashed')
plt.savefig('yeast.png')
plt.clf()
plt.close()

------------------------------------------------------------
Diversity-based Ensemble Classifier
------------------------------------------------------------


C:\Users\Alan\Anaconda2\envs\python3\lib\site-packages\sklearn\discriminant_analysis.py:706: RuntimeWarning: divide by zero encountered in power
  X2 = np.dot(Xm, R * (S ** (-0.5)))
C:\Users\Alan\Anaconda2\envs\python3\lib\site-packages\sklearn\discriminant_analysis.py:706: RuntimeWarning: invalid value encountered in multiply
  X2 = np.dot(Xm, R * (S ** (-0.5)))
C:\Users\Alan\Anaconda2\envs\python3\lib\site-packages\sklearn\discriminant_analysis.py:709: RuntimeWarning: divide by zero encountered in log
  u = np.asarray([np.sum(np.log(s)) for s in self.scalings_])
C:\Users\Alan\Anaconda2\envs\python3\lib\site-packages\sklearn\discriminant_analysis.py:706: RuntimeWarning: divide by zero encountered in power
  X2 = np.dot(Xm, R * (S ** (-0.5)))
C:\Users\Alan\Anaconda2\envs\python3\lib\site-packages\sklearn\discriminant_analysis.py:706: RuntimeWarning: invalid value encountered in multiply
  X2 = np.dot(Xm, R * (S ** (-0.5)))
C:\Users\Alan\Anaconda2\envs\python3\lib\site-packages\sklearn\

C:\Users\Alan\Anaconda2\envs\python3\lib\site-packages\sklearn\discriminant_analysis.py:706: RuntimeWarning: divide by zero encountered in power
  X2 = np.dot(Xm, R * (S ** (-0.5)))
C:\Users\Alan\Anaconda2\envs\python3\lib\site-packages\sklearn\discriminant_analysis.py:706: RuntimeWarning: invalid value encountered in multiply
  X2 = np.dot(Xm, R * (S ** (-0.5)))
C:\Users\Alan\Anaconda2\envs\python3\lib\site-packages\sklearn\discriminant_analysis.py:709: RuntimeWarning: divide by zero encountered in log
  u = np.asarray([np.sum(np.log(s)) for s in self.scalings_])
C:\Users\Alan\Anaconda2\envs\python3\lib\site-packages\sklearn\discriminant_analysis.py:706: RuntimeWarning: divide by zero encountered in power
  X2 = np.dot(Xm, R * (S ** (-0.5)))
C:\Users\Alan\Anaconda2\envs\python3\lib\site-packages\sklearn\discriminant_analysis.py:706: RuntimeWarning: invalid value encountered in multiply
  X2 = np.dot(Xm, R * (S ** (-0.5)))
C:\Users\Alan\Anaconda2\envs\python3\lib\site-packages\sklearn\

C:\Users\Alan\Anaconda2\envs\python3\lib\site-packages\sklearn\discriminant_analysis.py:706: RuntimeWarning: divide by zero encountered in power
  X2 = np.dot(Xm, R * (S ** (-0.5)))
C:\Users\Alan\Anaconda2\envs\python3\lib\site-packages\sklearn\discriminant_analysis.py:706: RuntimeWarning: invalid value encountered in multiply
  X2 = np.dot(Xm, R * (S ** (-0.5)))
C:\Users\Alan\Anaconda2\envs\python3\lib\site-packages\sklearn\discriminant_analysis.py:709: RuntimeWarning: divide by zero encountered in log
  u = np.asarray([np.sum(np.log(s)) for s in self.scalings_])
C:\Users\Alan\Anaconda2\envs\python3\lib\site-packages\sklearn\discriminant_analysis.py:706: RuntimeWarning: divide by zero encountered in power
  X2 = np.dot(Xm, R * (S ** (-0.5)))
C:\Users\Alan\Anaconda2\envs\python3\lib\site-packages\sklearn\discriminant_analysis.py:706: RuntimeWarning: invalid value encountered in multiply
  X2 = np.dot(Xm, R * (S ** (-0.5)))
C:\Users\Alan\Anaconda2\envs\python3\lib\site-packages\sklearn\

C:\Users\Alan\Anaconda2\envs\python3\lib\site-packages\sklearn\discriminant_analysis.py:706: RuntimeWarning: divide by zero encountered in power
  X2 = np.dot(Xm, R * (S ** (-0.5)))
C:\Users\Alan\Anaconda2\envs\python3\lib\site-packages\sklearn\discriminant_analysis.py:706: RuntimeWarning: invalid value encountered in multiply
  X2 = np.dot(Xm, R * (S ** (-0.5)))
C:\Users\Alan\Anaconda2\envs\python3\lib\site-packages\sklearn\discriminant_analysis.py:709: RuntimeWarning: divide by zero encountered in log
  u = np.asarray([np.sum(np.log(s)) for s in self.scalings_])
C:\Users\Alan\Anaconda2\envs\python3\lib\site-packages\sklearn\discriminant_analysis.py:706: RuntimeWarning: divide by zero encountered in power
  X2 = np.dot(Xm, R * (S ** (-0.5)))
C:\Users\Alan\Anaconda2\envs\python3\lib\site-packages\sklearn\discriminant_analysis.py:706: RuntimeWarning: invalid value encountered in multiply
  X2 = np.dot(Xm, R * (S ** (-0.5)))
C:\Users\Alan\Anaconda2\envs\python3\lib\site-packages\sklearn\

...

C:\Users\Alan\Anaconda2\envs\python3\lib\site-packages\sklearn\discriminant_analysis.py:706: RuntimeWarning: divide by zero encountered in power
  X2 = np.dot(Xm, R * (S ** (-0.5)))
C:\Users\Alan\Anaconda2\envs\python3\lib\site-packages\sklearn\discriminant_analysis.py:706: RuntimeWarning: invalid value encountered in multiply
  X2 = np.dot(Xm, R * (S ** (-0.5)))
C:\Users\Alan\Anaconda2\envs\python3\lib\site-packages\sklearn\discriminant_analysis.py:709: RuntimeWarning: divide by zero encountered in log
  u = np.asarray([np.sum(np.log(s)) for s in self.scalings_])
C:\Users\Alan\Anaconda2\envs\python3\lib\site-packages\sklearn\discriminant_analysis.py:706: RuntimeWarning: divide by zero encountered in power
  X2 = np.dot(Xm, R * (S ** (-0.5)))
C:\Users\Alan\Anaconda2\envs\python3\lib\site-packages\sklearn\discriminant_analysis.py:706: RuntimeWarning: invalid value encountered in multiply
  X2 = np.dot(Xm, R * (S ** (-0.5)))
C:\Users\Alan\Anaconda2\envs\python3\lib\site-packages\sklearn\

C:\Users\Alan\Anaconda2\envs\python3\lib\site-packages\sklearn\discriminant_analysis.py:706: RuntimeWarning: divide by zero encountered in power
  X2 = np.dot(Xm, R * (S ** (-0.5)))
C:\Users\Alan\Anaconda2\envs\python3\lib\site-packages\sklearn\discriminant_analysis.py:706: RuntimeWarning: invalid value encountered in multiply
  X2 = np.dot(Xm, R * (S ** (-0.5)))
C:\Users\Alan\Anaconda2\envs\python3\lib\site-packages\sklearn\discriminant_analysis.py:709: RuntimeWarning: divide by zero encountered in log
  u = np.asarray([np.sum(np.log(s)) for s in self.scalings_])
C:\Users\Alan\Anaconda2\envs\python3\lib\site-packages\sklearn\discriminant_analysis.py:706: RuntimeWarning: divide by zero encountered in power
  X2 = np.dot(Xm, R * (S ** (-0.5)))
C:\Users\Alan\Anaconda2\envs\python3\lib\site-packages\sklearn\discriminant_analysis.py:706: RuntimeWarning: invalid value encountered in multiply
  X2 = np.dot(Xm, R * (S ** (-0.5)))
C:\Users\Alan\Anaconda2\envs\python3\lib\site-packages\sklearn\

C:\Users\Alan\Anaconda2\envs\python3\lib\site-packages\sklearn\discriminant_analysis.py:706: RuntimeWarning: divide by zero encountered in power
  X2 = np.dot(Xm, R * (S ** (-0.5)))
C:\Users\Alan\Anaconda2\envs\python3\lib\site-packages\sklearn\discriminant_analysis.py:706: RuntimeWarning: invalid value encountered in multiply
  X2 = np.dot(Xm, R * (S ** (-0.5)))
C:\Users\Alan\Anaconda2\envs\python3\lib\site-packages\sklearn\discriminant_analysis.py:709: RuntimeWarning: divide by zero encountered in log
  u = np.asarray([np.sum(np.log(s)) for s in self.scalings_])
C:\Users\Alan\Anaconda2\envs\python3\lib\site-packages\sklearn\discriminant_analysis.py:706: RuntimeWarning: divide by zero encountered in power
  X2 = np.dot(Xm, R * (S ** (-0.5)))
C:\Users\Alan\Anaconda2\envs\python3\lib\site-packages\sklearn\discriminant_analysis.py:706: RuntimeWarning: invalid value encountered in multiply
  X2 = np.dot(Xm, R * (S ** (-0.5)))
C:\Users\Alan\Anaconda2\envs\python3\lib\site-packages\sklearn\

C:\Users\Alan\Anaconda2\envs\python3\lib\site-packages\sklearn\discriminant_analysis.py:706: RuntimeWarning: divide by zero encountered in power
  X2 = np.dot(Xm, R * (S ** (-0.5)))
C:\Users\Alan\Anaconda2\envs\python3\lib\site-packages\sklearn\discriminant_analysis.py:706: RuntimeWarning: invalid value encountered in multiply
  X2 = np.dot(Xm, R * (S ** (-0.5)))
C:\Users\Alan\Anaconda2\envs\python3\lib\site-packages\sklearn\discriminant_analysis.py:709: RuntimeWarning: divide by zero encountered in log
  u = np.asarray([np.sum(np.log(s)) for s in self.scalings_])
C:\Users\Alan\Anaconda2\envs\python3\lib\site-packages\sklearn\discriminant_analysis.py:706: RuntimeWarning: divide by zero encountered in power
  X2 = np.dot(Xm, R * (S ** (-0.5)))
C:\Users\Alan\Anaconda2\envs\python3\lib\site-packages\sklearn\discriminant_analysis.py:706: RuntimeWarning: invalid value encountered in multiply
  X2 = np.dot(Xm, R * (S ** (-0.5)))
C:\Users\Alan\Anaconda2\envs\python3\lib\site-packages\sklearn\

ValueError: y has only 1 sample in class 1, covariance is ill defined.

In [ ]:
d25 = compare_results(data=spam.iloc[:, :-1].values, target=spam.iloc[:, -1].values, size=25)
d50 = compare_results(data=spam.iloc[:, :-1].values, target=spam.iloc[:, -1].values, size=50)
d100 = compare_results(data=spam.iloc[:, :-1].values, target=spam.iloc[:, -1].values, size=100)

plt.plot(range(1, 101), d25, label='25 cromossomos')
plt.plot(range(1, 101), d50, label='50 cromossomos')
plt.plot(range(1, 101), d100, label='100 cromossomos')
plt.legend()
plt.ylabel('Diversidade')
plt.xlabel('Nº de épocas')
plt.grid(ls='dashed')
plt.savefig('spam.png')
plt.clf()
plt.close()

------------------------------------------------------------
Diversity-based Ensemble Classifier
------------------------------------------------------------
Accuracy : 0.7684039087947883
F1-score : 0.5886980455891058
Precision: 0.8610153167004378
Recall   : 0.4860016059630662
ROC AUC  : 0.7170294642575683
DEC done in:  9712588.9 ms
------------------------------------------------------------
Genetic-based Ensemble Classifier
------------------------------------------------------------
